In [1]:
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Deflategate (Chapter 12.2 in Inferential Thinking)
On January 18, 2015, the Indianapolis Colts and the New England Patriots played the American Football Conference (AFC) championship game to determine which would play in the Super Bowl. After the game, there were allegations that the Patriots’ footballs had not been inflated as much as the regulations required; they were softer. This could be an advantage, as softer balls might be easier to catch.

For several weeks, the world of American football was consumed by accusations, denials, theories, and suspicions: the press labeled the topic Deflategate, after the Watergate political scandal of the 1970’s. The National Football League (NFL) commissioned an independent analysis.

Pressure is often measured in pounds per square inch (psi). NFL rules stipulate that game balls must be inflated to have pressures in the range 12.5 - 13.5 psi. Each team plays with 12 balls. Teams have the responsibility of maintaining the pressure in their own footballs, but game officials inspect the balls. Before the start of the AFC game, all the Patriots’ balls were at about 12.5 psi. Most of the Colts’ balls were at about 13.0 psi. However, these pre-game data were not recorded.

During the second quarter, the Colts intercepted a Patriots ball. On the sidelines, they measured the pressure of the ball and determined that it was below the 12.5 psi threshold. Promptly, they informed officials.

At half-time, all the game balls were collected for inspection. Two officials, Clete Blakeman and Dyrol Prioleau, measured the pressure in each of the balls.

The Patriots ball that had been intercepted by the Colts was not inspected at half-time. Nor were most of the Colts’ balls – the officials simply ran out of time and had to relinquish the balls for the start of second half play.

Below, each row corresponds to one football. Pressure was measured in psi.

In [2]:
football = Table.read_table('deflategate.csv')
football.show()

Team,Blakeman,Prioleau
Patriots,11.5,11.8
Patriots,10.85,11.2
Patriots,11.15,11.5
Patriots,10.7,11
Patriots,11.1,11.45
Patriots,11.6,11.95
Patriots,11.85,12.3
Patriots,11.1,11.55
Patriots,10.95,11.35
Patriots,10.5,10.9


For each of the 15 balls that were inspected, the two officials got different results. It is not uncommon that repeated measurements on the same object yield different results, especially when the measurements are performed by different people. So we will assign to each the ball the **average** of the 2 measurements made on that ball.

In [3]:
# Calculate the average of each row
average = (football.column(1) + football.column(2))/2
# Update the 'football' table
football = football.drop(1, 2).with_column('Combined', average)
football

Team,Combined
Patriots,11.65
Patriots,11.025
Patriots,11.325
Patriots,10.85
Patriots,11.275
Patriots,11.775
Patriots,12.075
Patriots,11.325
Patriots,11.15
Patriots,10.7


At a glance, it seems Patriots’ footballs had lower pressure than the Colts’ balls. Because some deflation is normal during the course of a game, the independent analysts decided to calculate the drop in pressure from the start of the game. Recall that the Patriots’ balls had all started out at about 12.5 psi, and the Colts’ balls at about 13.0 psi. Therefore the drop in pressure for the Patriots’ balls was computed as 12.5 minus the pressure at half-time, and the drop in pressure for the Colts’ balls was 13.0 minus the pressure at half-time.

We can calculate the pressure drop for each football, by first setting up an array of the starting values. For this we will need:
* An array of 11 elements, all of them is 12.5
* Another array of 4 elements, all of them is 13

We will use the NumPy function `np.ones`, which takes a count as its argument and returns an array of that many elements, each of which is 1.

In [4]:
np.ones(11)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [5]:
patriots_start = 12.5 * np.ones(11)
colts_start = 13 * np.ones(4)
start = np.append(patriots_start, colts_start)
start

array([12.5, 12.5, 12.5, 12.5, 12.5, 12.5, 12.5, 12.5, 12.5, 12.5, 12.5,
       13. , 13. , 13. , 13. ])

Now we can subtract the starting pressure with the average measurement to obtain the pressure drop.

In [6]:
# Subtract each starting value with 
drop = start - football.column('Combined')
football = football.with_column('Pressure Drop', drop)
football.show()

Team,Combined,Pressure Drop
Patriots,11.65,0.85
Patriots,11.025,1.475
Patriots,11.325,1.175
Patriots,10.85,1.65
Patriots,11.275,1.225
Patriots,11.775,0.725
Patriots,12.075,0.425
Patriots,11.325,1.175
Patriots,11.15,1.35
Patriots,10.7,1.8


In [7]:
# Random fact: you can do the following
['Patriots'] * 11 + ['Colts'] * 4

['Patriots',
 'Patriots',
 'Patriots',
 'Patriots',
 'Patriots',
 'Patriots',
 'Patriots',
 'Patriots',
 'Patriots',
 'Patriots',
 'Patriots',
 'Colts',
 'Colts',
 'Colts',
 'Colts']

It looks as though the Patriots’ drops were larger than the Colts’. Let’s look at the average drop in each of the two groups.

In [8]:
football = football.drop('Combined')
average_drop = football.group('Team', np.average)
average_drop

Team,Pressure Drop average
Colts,0.46875
Patriots,1.20227


The average drop for the Patriots was about 1.2 psi compared to about 0.47 psi for the Colts.

The question now is why the Patriots’ footballs had a larger drop in pressure, on average, than the Colts footballs. Could it be due to chance?

## The Hypotheses
How does chance come in here? Nothing was being selected at random. But we can make a chance model by hypothesizing that **the 11 Patriots’ drops look like a random sample of 11 out of all the 15 drops, with the Colts’ drops being the remaining four**. That’s a completely specified chance model under which we can simulate data. So it’s the null hypothesis.

For the alternative, we can take the position that **the Patriots’ drops are too large, on average, to resemble a random sample drawn from all the drops**.

### Test Statistic
A natural statistic is **the difference between the 2 average drops**, which is **average drops of Patriots - average drop of Colts**. Large value of the test statistic will favor the alternative hypothesis.

In [9]:
average_drop_array = average_drop.column(1)
observed_difference = average_drop_array.item(1) - average_drop_array.item(0)
observed_difference

0.733522727272728

The positive difference above shows that the average drop in pressure of the Patriots' ball was indeed greater than that of the Colts.

### How do we draw more samples under the null hypothesis? (Vid Lecture)

## Comparing 2 Samples
We need a method of comparing 2 samples. In our case, we want to know whether our 2 samples look different.

### Permutation
We're going to do a permutation test. 

If the Patriots' balls and the Colts' balls came from the same distribution (roughly same pressure drop, same average, same variance), 
<img src = 'permutation.jpg' width = 400/>
* Patriot's balls = dark blue
* Colts' balls = orange

then it should not matter if the order is shuffled.

## Permutation Test
* Whether 2 samples are drawn randomly from the same underlying distribution
    * Attribute distribution are the same for both classes
* If the **null** is true, all rearrangements of the attribute values among the 2 classes are equally likely
* Compute the observed test statistic
    * Then shuffle the attribute values and recompute the statistic
    * Repeat
    * Compare with observed statistic

Currently we have the `football` table containing the pressure drops.

In [10]:
football

Team,Pressure Drop
Patriots,0.85
Patriots,1.475
Patriots,1.175
Patriots,1.65
Patriots,1.225
Patriots,0.725
Patriots,0.425
Patriots,1.175
Patriots,1.35
Patriots,1.8


Below, we shuffle the football table without replacement

In [11]:
football.sample(with_replacement = False)

Team,Pressure Drop
Patriots,1.175
Patriots,1.8
Patriots,1.225
Colts,0.475
Patriots,1.35
Patriots,1.65
Colts,0.275
Colts,0.475
Colts,0.65
Patriots,0.425


However, the problem with above is that the arrangement of the **Team** column is also shuffled. Instead, we only want to shuffle the pressure drops and assign them to the unchanged arrangement of the **Team**.

In [19]:
shuffled_drops = football.select('Pressure Drop').sample(with_replacement = False)
shuffled_drops

Pressure Drop
1.8
0.65
0.475
1.475
0.475
1.175
1.175
1.225
0.85
1.65


In [21]:
football.select('Team').with_column('Drop', shuffled_drops.column(0))

Team,Drop
Patriots,1.8
Patriots,0.65
Patriots,0.475
Patriots,1.475
Patriots,0.475
Patriots,1.175
Patriots,1.175
Patriots,1.225
Patriots,0.85
Patriots,1.65
